# IR spectral analysis of organic and coordination compounds via machine learning and neural networks
By: Koshelev Daniil 

-Work in progress (last updated:04/04/2023)-

B. Цель данного блокнота

Целью этого блокнота является получение базовой точности на основе анализа пиков только для ближайшего положения по тиблице из википедии.

C. План реализации

1. Импортировать данные и модель
2. Реализовать расчёт пиков.
3. Ипортировать данные по положению референсных сигналов.
4. Получить предсказание на основе одного ближайшего соседа.
5. Сравнить с результатами предсказания модели.

NOTES
1. Импортированы данные и модель
2. Реализован эмпирически алгоритм
3. Оценена точность эмпирического алгоритма


Клонирование данных с GitHub. На сайте выложены как исходные списки веществ, так и скачанные спектры соединений, которые были запарсены кодом ниже.

In [9]:
!git clone https://github.com/Lamblador/IR_expert_system.git

fatal: destination path 'IR_expert_system' already exists and is not an empty directory.


In [10]:
# Установка нужных пакетов
from IPython.core.display import clear_output
!pip install --upgrade jcamp
!pip install torchmetrics
!pip install shap
!pip install peakutils
clear_output()

In [11]:
# Вводятся нужные импорты
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os
import numpy as np
import torch
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import shap
import torch.nn as nn
from peakutils.peak import indexes as index_utils


In [12]:
import random
import torch
import torch.nn as nn
from torchvision import datasets, transforms


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_random_seed(42)

# Импорт табличных данных

In [13]:
labels_df = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_inchi_group_list.csv')
labels_df = labels_df.set_index("CAS")

In [14]:
#Считывание датасета
def read_dataset(dataset_name):
  '''
  Function for reading dataset into pandas Dataframe from /IR_expert_system/data/datasets/

  dataset_name - name of dataset's folder to read

  return pd.Dataframe with whole dataset

  parameters:
      name = ['dataset3']
          dataset3 - dataset made from reading most of jcam files, with padded y, interpolated x and peak layer. Saved in 10 pieces. 
  '''
  df_list = []
  # choose current directory
  curr_directory = os.getcwd()
  os.chdir(curr_directory)  # set to current directory
  files = glob.glob('/content/IR_expert_system/data/datasets/'+dataset_name+'/*.pkl')  # look for files without extension. Returns a list.
  # Try to read the file. If can't be read, skip it.
  #print(files)
  for i, file in tqdm(enumerate(files)):
      df = pd.read_pickle(file)
      df_list.append(df)
  #print(df_list)
  result = pd.concat(df_list)
  return result

dataset_df1 = read_dataset('dataset3')
dataset_df1

10it [00:03,  2.58it/s]


,index,title,jcamp-dx,data type,origin,owner,molform,$nist source,spectrometer/data system,state,...,number of interferograms averaged per single channel spectrum,sample description,temperature,$spectra version,$uncertainty in y,pressure,x1,y1,y_peak,x_peak
cas registry no,,,,,,,,,,,,,,,,,,,,,
7496-45-9,7496-45-9,"Purine, 1-ethyl-",4.24,INFRARED SPECTRUM,SOUTHERN RESEARCH INSTITUTE,Copyright (C) 1987 by the Coblentz Society\nCo...,C7 H8 N4,COBLENTZ,PERKIN-ELMER 221 (GRATING),SOLID (0.8 mg / 650 mg KBr DISC),...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.13739997148513794, 0.13739997148513794, 0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[580, 640, 664, 726, 798, 834, 928, 966, 1022,..."
106-36-5,106-36-5,"Propanoic acid, propyl ester",4.24,INFRARED SPECTRUM,Sadtler Research Labs Under US-EPA Contract,NIST Standard Reference Data Program\nCollecti...,C 6 H 12 O 2,MSDC-IR,NaN,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.05554400011897087, 0.054496001452207565, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[542, 594, 618, 642, 674, 770, 814, 838, 882, ..."
5347-82-0,5347-82-0,DIMETHANE SULFONAMIDE,4.24,INFRARED SPECTRUM,"WYANDOTTE CHEMICALS CORP., WYANDOTTE, MICHIGAN...",COBLENTZ SOCIETY\nCollection (C) 2018 copyrigh...,C2 H7 N O4 S2,COBLENTZ,"Not specified, most likely a prism, grating, o...",SOLID (MULL),...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.21799999475479126, 0.21799999475479126, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[632, 662, 766, 866, 974, 1024, 1066, 1150, 13..."
112-17-4,112-17-4,1-Decanol acetate,4.24,INFRARED SPECTRUM,NIST Mass Spectrometry Data Center,NIST Standard Reference Data Program\nCollecti...,C 12 H 24 O 2,MSDC-IR,HP-GC/MS/IRD,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0001320000010309741, 0.0001320000010309741,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1042, 1234, 1370, 1462, 1762, 2866, 2934]"
4919-37-3,4919-37-3,"4-Hydroxy-3,5-dimethylbenzoic acid",4.24,INFRARED SPECTRUM,EPA-IR VAPOR PHASE LIBRARY,SRD/NIST\nCollection (C) 2018 copyright by the...,C 9 H 10 O 3,MSDC-IR,NaN,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.01228999998420477, 0.01228999998420477, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[588, 656, 770, 934, 1104, 1146, 1198, 1266, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25173-72-2,25173-72-2,"3-(3,4,5-Trimethoxyphenyl)propionic acid",4.24,INFRARED SPECTRUM,EPA-IR VAPOR PHASE LIBRARY,SRD/NIST\nCollection (C) 2018 copyright by the...,C 12 H 16 O 5,MSDC-IR,NaN,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.010820000432431698, 0.010820000432431698, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[576, 778, 832, 930, 1016, 1044, 1118, 1186, 1..."
56946-83-9,56946-83-9,"2,5-Dichlorothiophene-3-sulphonyl chloride",4.24,INFRARED SPECTRUM,EPA-IR VAPOR PHASE LIBRARY,SRD/NIST\nCollection (C) 2018 copyright by the...,C 4 H Cl 3 O 2 S 2,MSDC-IR,NaN,gas,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.13794000446796417, 0.13794000446796417, 0.1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[558, 598, 632, 830, 854, 1008, 1046, 1136, 11..."
823-09-6,823-09-6,"Pyrimidine, 2-(methylthio)-,",4.24,INFRARED SPECTRUM,SOUTHERN RESEARCH INSTITUTE,Copyright (C) 1987 by the Coblentz Society\nCo...,C5 H6 N2 S,COBLENTZ,PERKIN-ELMER 21 (GRATING),LIQUID (NEAT),...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.2558000087738037, 0.2558000087738037, 0.255...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[610, 710, 748, 770, 796, 876, 964, 1012, 1062..."


In [15]:
dataset_df_labeled = labels_df.join(dataset_df1)

In [16]:
dataset_df_labeled1 = dataset_df_labeled[['Name', 'Formula', 'Mw', 'Elements', 'InChi', 'SMILES', 'alkane',
       'methyl', 'alkene', 'alkyne', 'alcohols', 'amines', 'nitriles',
       'aromatics', 'alkyl halides', 'esters', 'ketones', 'aldehydes',
       'carboxylic acids', 'ether', 'acyl halides', 'amides', 'nitro',
       'title',
       'molform',
       'x', 'y', 'x1', 'y1', 'y_peak', 'x_peak']]
#dataset_df_labeled1

In [17]:
spectra_x =dataset_df_labeled.iloc[:,-1]
spectra_x_t = torch.Tensor(spectra_x[0])
#spectra_x_t

# Создание таблицы всех возможных колебаний

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def create_all_bonds_types_table():
  # Request information from wikipedia
  res = requests.get(
        "https://en.wikipedia.org/wiki/Infrared_spectroscopy_correlation_table")

  # Create a BeautifulSoup class
  soup = BeautifulSoup(res.content, 'lxml')

  # Extract info. from a table
  table = soup.find_all(class_="wikitable")[0]

  # Load table info. into a Pandas DataFrame
  df = pd.read_html(str(table), header=0)[0]
  return df

In [19]:
all_bonds_types_table = create_all_bonds_types_table()
all_bonds_types_table

,Bond,Type of bond,Specific type of bond,Absorption peak (cm−1),Appearance
0,C─H,alkyl,methyl,1260,strong
1,C─H,alkyl,methyl,1380,weak
2,C─H,alkyl,methyl,2870,medium to strong
3,C─H,alkyl,methyl,2960,medium to strong
4,C─H,alkyl,methylene,1470,strong
...,...,...,...,...,...
87,N─O,nitro compounds,aromatic,1520,lower if conjugated
88,N─O,nitro compounds,aromatic,1350,lower if conjugated
89,P─C,Organophosphorus compound,aromatic,1440-1460,medium
90,P─O,phosphorus oxide,bonded,1195-1250,strong


In [20]:
num_list = []
for i in range(len(all_bonds_types_table['Absorption peak (cm−1)'])):
  temp = all_bonds_types_table['Absorption peak (cm−1)'][i]
  try:
    num_list.append(int(temp))
  except: 
    
    if ">" in temp: 
        num = temp[1:]
        num = int(num)
        num_list.append(num)
    elif '–' in temp: 
        num_list1 = temp.split('–')
        num = (int(num_list1[0])+int(num_list1[1]))/2
        num_list.append(num)
    elif '-' in temp: 
        num_list2 = temp.split('-')
        num = (int(num_list2[0])+int(num_list2[1]))/2
        num_list.append(num)
    elif '~' in temp: 
        num = temp[1:]
        num = int(num)
        num_list.append(num)     
print(num_list)
all_bonds_types_table['Absorption peak (cm−1) num'] = num_list

[1260, 1380, 2870, 2960, 1470, 2850, 2925, 2890, 900, 2975, 3080, 3020, 900, 990, 685.0, 965, 820.0, 3070, 725.0, 700.0, 750, 775.0, 880.0, 830.0, 3300, 2720, 2820, 1645, 1655, 1660, 1675, 1670, 1600, 1650, 1625, 1600, 1660.0, 1450, 1500, 1580, 1600, 2120.0, 2225.0, 1720, 1685, 1685, 1750, 1775, 1725, 1710, 1685.0, 1735, 1760, 1820, 1800, 1650, 1580.0, 1580.0, 3640.0, 3300.0, 3530.0, 3000, 3450.0, 1600.0, 3000, 2800.0, 1050.0, 1100, 1175.0, 1200, 1120, 1240.0, 1275.0, 1200.0, 1120.0, 1657.5, 2250, 2230, 2137.5, 2065.0, 1050.0, 1150.0, 650.0, 550.0, 500, 1540, 1380, 1520, 1350, 1450.0, 1222.5, 1275.0]


In [21]:
print(all_bonds_types_table.to_string())

   Bond                Type of bond               Specific type of bond Absorption peak (cm−1)                                                          Appearance  Absorption peak (cm−1) num
0   C─H                       alkyl                              methyl                   1260                                                              strong                      1260.0
1   C─H                       alkyl                              methyl                   1380                                                                weak                      1380.0
2   C─H                       alkyl                              methyl                   2870                                                    medium to strong                      2870.0
3   C─H                       alkyl                              methyl                   2960                                                    medium to strong                      2960.0
4   C─H                       alkyl          

In [22]:
bond_type_cls_list = []
for i in range(len(all_bonds_types_table)):
  bond_type_cls = ""
  for st in list(all_bonds_types_table.iloc[i])[:-1]:
    bond_type_cls += str(st)+' '
  bond_type_cls_list.append(bond_type_cls)
print(bond_type_cls_list)

['C─H alkyl methyl 1260 strong ', 'C─H alkyl methyl 1380 weak ', 'C─H alkyl methyl 2870 medium to strong ', 'C─H alkyl methyl 2960 medium to strong ', 'C─H alkyl methylene 1470 strong ', 'C─H alkyl methylene 2850 medium to strong ', 'C─H alkyl methylene 2925 medium to strong ', 'C─H alkyl methine 2890 weak ', 'C─H vinyl C═CH2 900 strong ', 'C─H vinyl C═CH2 2975 medium ', 'C─H vinyl C═CH2 3080 medium ', 'C─H vinyl C═CH 3020 medium ', 'C─H vinyl monosubstituted alkenes 900 strong ', 'C─H vinyl monosubstituted alkenes 990 strong ', 'C─H vinyl cis-disubstituted alkenes 670–700 strong ', 'C─H vinyl trans-disubstituted alkenes 965 strong ', 'C─H vinyl trisubstituted alkenes 800–840 strong to medium ', 'C─H aromatic benzene/sub. benzene 3070 weak ', 'C─H aromatic monosubstituted benzene 700–750 strong ', 'C─H aromatic monosubstituted benzene 690–710 strong ', 'C─H aromatic ortho-disub. benzene 750 strong ', 'C─H aromatic meta-disub. benzene 750–800 strong ', 'C─H aromatic meta-disub. benzene 

In [23]:
for i in range(len(bond_type_cls_list)):
  print("'" + bond_type_cls_list[i]+"': ," )

'C─H alkyl methyl 1260 strong ': ,
'C─H alkyl methyl 1380 weak ': ,
'C─H alkyl methyl 2870 medium to strong ': ,
'C─H alkyl methyl 2960 medium to strong ': ,
'C─H alkyl methylene 1470 strong ': ,
'C─H alkyl methylene 2850 medium to strong ': ,
'C─H alkyl methylene 2925 medium to strong ': ,
'C─H alkyl methine 2890 weak ': ,
'C─H vinyl C═CH2 900 strong ': ,
'C─H vinyl C═CH2 2975 medium ': ,
'C─H vinyl C═CH2 3080 medium ': ,
'C─H vinyl C═CH 3020 medium ': ,
'C─H vinyl monosubstituted alkenes 900 strong ': ,
'C─H vinyl monosubstituted alkenes 990 strong ': ,
'C─H vinyl cis-disubstituted alkenes 670–700 strong ': ,
'C─H vinyl trans-disubstituted alkenes 965 strong ': ,
'C─H vinyl trisubstituted alkenes 800–840 strong to medium ': ,
'C─H aromatic benzene/sub. benzene 3070 weak ': ,
'C─H aromatic monosubstituted benzene 700–750 strong ': ,
'C─H aromatic monosubstituted benzene 690–710 strong ': ,
'C─H aromatic ortho-disub. benzene 750 strong ': ,
'C─H aromatic meta-disub. benzene 750–800 str

In [24]:
#Создание загрузчиков
# Number of classes
num_classes = len(bond_type_cls_list)
num_classes

92

In [ ]:
cls2indx_bonds = {}
indx2cls_bonds = {}
for i, classs in enumerate(bond_type_cls_list):
  cls2indx_bonds[f'{classs}']= i
  indx2cls_bonds[i] = classs
#print(cls2indx_bonds)
#print(indx2cls_bonds)

# Соотнесение сигналов с ближайшим соседом

In [26]:
print(num_list)
dataset_df_labeled1['x_peak'][0]

[1260, 1380, 2870, 2960, 1470, 2850, 2925, 2890, 900, 2975, 3080, 3020, 900, 990, 685.0, 965, 820.0, 3070, 725.0, 700.0, 750, 775.0, 880.0, 830.0, 3300, 2720, 2820, 1645, 1655, 1660, 1675, 1670, 1600, 1650, 1625, 1600, 1660.0, 1450, 1500, 1580, 1600, 2120.0, 2225.0, 1720, 1685, 1685, 1750, 1775, 1725, 1710, 1685.0, 1735, 1760, 1820, 1800, 1650, 1580.0, 1580.0, 3640.0, 3300.0, 3530.0, 3000, 3450.0, 1600.0, 3000, 2800.0, 1050.0, 1100, 1175.0, 1200, 1120, 1240.0, 1275.0, 1200.0, 1120.0, 1657.5, 2250, 2230, 2137.5, 2065.0, 1050.0, 1150.0, 650.0, 550.0, 500, 1540, 1380, 1520, 1350, 1450.0, 1222.5, 1275.0]


[530, 742, 850, 1014, 1102, 1174, 1274, 1350, 1422, 1482, 1542, 1602]

In [27]:
def det_nearest_peak(value, peak_list):
  return min(peak_list, key=lambda x: abs(x - value))

In [28]:
def det_nearest_peak_indx(value_min, peak_list):
  return peak_list.index(value_min)

In [29]:
print(det_nearest_peak(560, num_list))
print(det_nearest_peak_indx(det_nearest_peak(560, num_list), num_list))

550.0
83


In [30]:
cls_serie = []
for peak_list in tqdm(dataset_df_labeled1['x_peak']):
  cls_list = []
  try:
    for value in peak_list:
      cls_list.append(det_nearest_peak_indx(det_nearest_peak(value, num_list), num_list))
    cls_serie.append(cls_list)
  except:
    cls_serie.append(np.nan)
print(cls_serie)

100%|██████████| 12233/12233 [00:59<00:00, 205.61it/s]


[[83, 20, 23, 13, 67, 68, 72, 88, 37, 4, 85, 32], [82, 14, 20, 23, 70, 68, 72, 88, 4, 85, 34, 62], [82, 20, 22, 67, 68, 72, 88, 37, 4, 38, 85, 32, 58], [84, 83, 82, 82, 14, 18, 20, 23, 22, 15, 13, 66, 67, 81, 68, 71, 72, 88, 1, 1, 4, 87, 39, 27, 44, 49, 52, 53, 53, 79, 79, 76, 76, 76, 76, 25, 25, 25, 25, 2, 6, 3, 10, 58, 58, 58], [82, 19, 20, 21, 16, 23, 22, 15, 13, 66, 67, 69, 0, 72, 88, 1, 37, 4, 39, 32, 29, 7, 61], [83, 23, 15, 66, 68, 0, 72, 88, 1, 37, 4, 38, 32, 49, 5, 3, 11], [8, 66, 68, 72, 1, 37, 34, 25, 6, 9, 10], [83, 67, 68, 1, 37, 2, 6, 9, 17], [83, 18, 16, 15, 66, 67, 68, 90, 88, 88, 37, 4, 87, 32, 49, 25, 26, 2, 7, 6, 61, 11, 10], [82, 19, 16, 22, 67, 68, 90, 88, 85, 32], [19, 20, 21, 16, 23, 8, 15, 13, 66, 67, 68, 71, 88, 4, 87, 32, 44, 43, 54, 53, 53, 42, 76, 76, 25, 25, 25, 65, 7, 9, 17, 24, 24, 62, 62, 60, 58, 58], [18, 16, 22, 13, 67, 68, 0, 88, 38, 85, 32], [81, 88, 4, 48, 2, 6], [82, 82, 14, 18, 21, 16, 16, 23, 22, 8, 15, 66, 67, 0, 1, 38, 39, 34, 28, 43, 25, 65, 2

In [32]:
cls_serie1 = pd.Series(cls_serie)
df = pd.DataFrame(cls_serie1, columns = ['peak_cls'])
dataset_df_labeled1.reset_index()
dataset_df = dataset_df1.reset_index()
dataset_df1 = dataset_df.join(df, lsuffix='x')
dataset_df1 = dataset_df1.set_index('cas registry no')


In [33]:
dataset_df1['peak_cls'].dropna()

cas registry no
7496-45-9       [83, 20, 23, 13, 67, 68, 72, 88, 37, 4, 85, 32]
106-36-5        [82, 14, 20, 23, 70, 68, 72, 88, 4, 85, 34, 62]
5347-82-0     [82, 20, 22, 67, 68, 72, 88, 37, 4, 38, 85, 32...
112-17-4      [84, 83, 82, 82, 14, 18, 20, 23, 22, 15, 13, 6...
4919-37-3     [82, 19, 20, 21, 16, 23, 22, 15, 13, 66, 67, 6...
                                    ...                        
527-69-5      [84, 16, 15, 66, 81, 90, 0, 88, 37, 37, 4, 87,...
92291-24-2                              [69, 72, 1, 4, 2, 6, 9]
5024-31-7     [82, 18, 21, 16, 8, 15, 13, 66, 81, 69, 71, 72...
93861-86-0    [18, 21, 23, 67, 68, 88, 4, 85, 39, 32, 7, 6, 17]
13290-74-9    [84, 83, 82, 14, 20, 23, 22, 8, 66, 68, 71, 88...
Name: peak_cls, Length: 11901, dtype: object

# Эмпирический алгоритм соответствия

In [34]:
#Создаём словарь в котором ключом будет метка класса (17), 
#а значениями, те связи или подстроки, 
#поиск которых даёт однознакое представление на присутствие этого класса в спектре

fuc_grp_bonds_types = {
                   'alkane':'C─H',
                   'methyl':'CH3',
                   'alkene':'C=C',
                   'alkyne':'C≡C',
                   'alcohols':'O-H',
                   'amines':'N─H',
                   'nitriles':'N≡C', 
                   'aromatics':'CC',
                   'alkyl halides':'CHal', 
                   'esters':'COOR',
                   'ketones':'CO',
                   'aldehydes':'CHO', 
                   'carboxylic acids':'COOH',
                   'ether': 'ROR',
                   'acyl halides':'COHal',
                   'amides':'CON',
                   'nitro':'NO2'}
bonds_types_strs = {
                  'C─H': ['alkyl','methylene', 'methyl'],
                  'CH3': ['methyl'],
                  'C=C': ['vinyl', 'alkenes', 'dienes'],
                  'C≡C': ['alkynes'],
                  'С=O': ['C=O','aldehyde', 'ketone', 'aldehydes'],
                  'O-H': ['O─H','alcohols', 'phenols'],
                  'COOH': ['carboxylic acids','carboxylates'],
                  'N─H': ['amines','ammonium'],
                  'N≡C': ['nitriles', 'C≡N', 'isocyanides', 'isothiocyanates'],
                  'CC': ['aromatic','benzene'],
                  'CHal': ['fluoroalkanes','chloroalkanes', 'bromoalkanes', 'iodoalkanes'],
                  'COOR': ['ethers'],
                  'CO': ['ketone'],
                  'CHO': ['aldehyde'],
                  'ROR': ['esters'],
                  'COHal': ['acyl halides'],
                  'CON': ['amides'],
                  'NO2': ['nitro'],
}


In [35]:
#Функция выдаёт список меток классов по набору индексов классов из 92
def find_cls_labels(peak_list, cls2indx_bonds=cls2indx_bonds, indx2cls_bonds=indx2cls_bonds, fuc_grp_bonds_types=fuc_grp_bonds_types, bonds_types_strs=bonds_types_strs):
    mol_func_grps = []
    bond_class_list = []
    for i in peak_list: bond_class_list.append(indx2cls_bonds[i])
    fuc_class_list = []
    for bond_str in bond_class_list:
      # bond_str = 'C─H alkyl methyl 1260 '
      for cls, key_words_list in bonds_types_strs.items():
            for key_word in key_words_list:
              if key_word in bond_str: fuc_class_list.append(cls)
    #Получился список подклассов
    #Составляем таблицу
    for _, func_struct in fuc_grp_bonds_types.items():
            contains_func_grp = 0
            if func_struct in fuc_class_list: contains_func_grp = 1
            mol_func_grps.append(contains_func_grp)
        
    return mol_func_grps
    

In [36]:
#Проверка функции
find_cls_labels(dataset_df1['peak_cls'].iloc[0])

[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]

In [37]:
cas_inchi_df = list(dataset_df1.index)
column_names = list(fuc_grp_bonds_types.keys())
target_df = pd.DataFrame(index = cas_inchi_df, columns = column_names)

In [38]:
#Iterate the rows
for ind, _ in enumerate(cas_inchi_df):
  try:
    target_df.iloc[ind, :] = find_cls_labels(dataset_df1['peak_cls'].iloc[ind])
  except:
    None
target_df.dropna()

,alkane,methyl,alkene,alkyne,alcohols,amines,nitriles,aromatics,alkyl halides,esters,ketones,aldehydes,carboxylic acids,ether,acyl halides,amides,nitro
7496-45-9,1,1,1,0,1,0,0,1,1,0,0,0,1,0,0,0,1
106-36-5,1,1,1,0,1,1,0,1,1,1,0,0,1,0,0,0,1
5347-82-0,1,1,1,0,1,0,0,1,1,0,0,0,1,0,0,0,1
112-17-4,1,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1
4919-37-3,1,1,1,0,1,0,0,1,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527-69-5,1,1,1,0,1,1,0,1,1,0,0,0,1,0,0,0,1
92291-24-2,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
5024-31-7,1,1,1,0,1,1,1,1,1,1,1,1,1,0,0,0,1
93861-86-0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1


# Оценка качества эмпирического алгоритма

In [39]:

classes = list(fuc_grp_bonds_types.keys())
y_true = []
y_pred = []
  
df = dataset_df_labeled1.iloc[:,6:23].dropna()
target_df1 = target_df.dropna()
for index, _ in target_df1.iterrows():
  try:
    y_true.append(torch.Tensor(df.loc[index]))
      
    y_pred.append(torch.Tensor(target_df.loc[index]))
  except:
      
      None
print(len(y_true), len(y_pred))
# print(y_true, y_pred)


9936 9936


In [40]:
from torch.nn.utils.rnn import pad_sequence
y_true1 = pad_sequence(y_true)
y_pred1 = pad_sequence(y_pred)
y_pred1.T.shape

torch.Size([9936, 17])

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_true1.T, y_pred1.T, target_names = classes))

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


                  precision    recall  f1-score   support

          alkane       0.68      0.96      0.80      6770
          methyl       0.62      0.96      0.75      6192
          alkene       0.14      0.92      0.24      1348
          alkyne       0.02      0.27      0.04       196
        alcohols       0.22      0.89      0.35      2167
          amines       0.13      0.30      0.18      1320
        nitriles       0.04      0.33      0.07       382
       aromatics       0.58      0.94      0.72      5825
   alkyl halides       0.25      0.72      0.38      2546
          esters       0.11      0.48      0.18      1166
         ketones       0.09      0.41      0.15       918
       aldehydes       0.02      0.50      0.04       206
carboxylic acids       0.06      0.75      0.11       589
           ether       0.24      0.09      0.13      2489
    acyl halides       0.01      0.12      0.02        93
          amides       0.00      0.00      0.00       701
           ni

Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.


# Создание класса Dataset

In [42]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

class IRSpectraDataset(Dataset):
    def __init__(self, dataset_df_labeled):
        self.dataset_df = dataset_df_labeled
        self.spectra_labels = self.dataset_df.iloc[:,6:23]
        self.spectra_x = self.dataset_df.iloc[:,-3]
        self.spectra_y = self.dataset_df.iloc[:,-2]
        self.spectra_y_peak = self.dataset_df.iloc[:,-1]
        
    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        
        spectra_x = torch.Tensor(self.spectra_x.iloc[idx]).reshape((1, 1800))
        spectra_y = torch.Tensor(self.spectra_y.iloc[idx]).reshape((1, 1800)) 
        spectra_y_peak = torch.Tensor(self.spectra_y_peak.iloc[idx]).reshape((1, 1800))     
        spectra = torch.cat((spectra_x, spectra_y, spectra_y_peak))
        
        label = torch.Tensor(self.spectra_labels.iloc[idx])
        #transform
        return spectra, label

In [43]:
dataset_ir_spectra = IRSpectraDataset(dataset_df_labeled1)

In [44]:
#Создание объекта класса и разбиение на трейн,тест и вал
from torch.utils.data import random_split
dataset_ir_spectra = IRSpectraDataset(dataset_df_labeled1.dropna())
train_size = int(0.7 * len(dataset_ir_spectra))
val_size = int(0.1 * len(dataset_ir_spectra))
test_size = len(dataset_ir_spectra) - train_size - val_size
train_dataset, test_dataset = random_split(dataset_ir_spectra, [train_size, int(test_size+val_size)])
test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

In [45]:
#Создание загрузчиков
# Number of classes
classes = list(dataset_ir_spectra.spectra_labels.columns)
num_classes = len(classes)
num_classes

17

In [ ]:
cls2inx = {}
inx2cls= {}
for i, classs in enumerate(classes):
  cls2inx[f'{classs}']= i
  inx2cls[i] = classs
#print(cls2inx)
#print(inx2cls)

In [ ]:
# Расчёт весов
wts4cls = {}
for classs in classes:
  wts4cls[classs] = dataset_ir_spectra.spectra_labels[classs].mean()
#print(wts4cls)

In [48]:
#Веса классов
wts_list = []
for name, values in wts4cls.items():
  wts_list.append(values)
weight_for_every_class = torch.tensor(wts_list)
print(weight_for_every_class)

tensor([0.6827, 0.6218, 0.1355, 0.0196, 0.2186, 0.1308, 0.0382, 0.5846, 0.2595,
        0.1164, 0.0941, 0.0206, 0.0603, 0.2501, 0.0096, 0.0709, 0.0573],
       dtype=torch.float64)


In [ ]:
# #Модифицирование весов для каждого класс:
# level = 0.3
# grain = 0.15
# lg = level/grain
# print(lg)
# #Классы меньше grain процентов - увеличиваются в level/20 раза 
# #Классы меньше 10 процентов - увеличиваются в level/10 раз 
# #Классы меньше 5 процентов - увеличиваются в level/5 раз
# #Классы меньше 1 процента - увеличиваются в level раз
# weight_for_every_class_modified = []
# for i in weight_for_every_class:
#   if i <= grain/8: weight_for_every_class_modified.append(i/(32/lg))
#   elif (i <= grain/4) and (i>grain/8): weight_for_every_class_modified.append(i/(16/lg))
#   elif (i <= grain/2) and (i>grain/4): weight_for_every_class_modified.append(i/(8/lg))
#   elif (i <= grain) and (i>grain/2): weight_for_every_class_modified.append(i/(4*lg))
#   else: weight_for_every_class_modified.append(i)
# weight_for_every_class_modified = torch.Tensor(weight_for_every_class_modified)
# print(weight_for_every_class)
# print(weight_for_every_class_modified)
# print(weight_for_every_class_modified.size())

In [50]:
#Расчёт веса образца на основе минимального веса любой функциональной группы 
labels1 = train_dataset.dataset.spectra_labels.to_numpy().copy()
labels1[labels1 == 0.0] = 100
#labels1

In [52]:
train_labels = torch.tensor(labels1)
weighted_labels  = train_labels*weight_for_every_class
weights_for_every_sample = []
for label_tensor in weighted_labels:
  weight = np.nanmin(label_tensor.numpy())
  weights_for_every_sample.append(weight)

weights_for_every_sample = torch.tensor(weights_for_every_sample)
print(weights_for_every_sample)

tensor([0.0573, 0.0573, 0.0573,  ..., 0.2595, 0.0573, 0.1308],
       dtype=torch.float64)


In [53]:
weights_train = weights_for_every_sample[:len(train_dataset)]

In [54]:
weights_train

tensor([0.0573, 0.0573, 0.0573,  ..., 0.0603, 0.2186, 0.2595],
       dtype=torch.float64)

In [55]:
#Проверка на положительные вероятности
for i in range(len(weights_train)):
    if weights_train[i] < 0:
        weights_train[i] = abs(weights_train[i])

In [56]:
weights_train[weights_train<0]

tensor([], dtype=torch.float64)

In [57]:
#Сумма весов должна быть равна единице. Вычисление суммы:
sum_weights = 0
for i in range(len(weights_train)):
 
  #print(weights_train[i])
  if weights_train[i] == torch.tensor(np.nan, dtype=torch.float64) : weights_train[i] = 1
  #print(weights_train[i])
  #print(sum_weights)
  sum_weights +=weights_train[i].numpy()
print(sum_weights)

1743.1378845343193


In [58]:
#Сумма весов должна быть равна единице. Нормирование на сумму.
for i in range(len(weights_train)):
    weights_train[i] =  weights_train[i]/sum_weights

In [59]:
weights_train

tensor([3.2878e-05, 3.2878e-05, 3.2878e-05,  ..., 3.4619e-05, 1.2542e-04,
        1.4887e-04], dtype=torch.float64)

In [60]:
#Даталоадер

from torch.utils.data import DataLoader, WeightedRandomSampler

# Batch size
batch_size = 128

dataloader_ir_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataloader_ir_val = DataLoader(val_dataset, batch_size=val_size, shuffle=False)
weightedsampler = WeightedRandomSampler(weights_train, len(weights_train)) 

dataloader_ir_train = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    sampler= weightedsampler
)

# Модель

Классификация по RF

In [61]:
dataset_df = dataset_df_labeled1.dropna()
spectra_labels = dataset_df.iloc[:,6:23]
spectra_y = dataset_df.iloc[:,-3]

In [62]:
spectra_labels1 = spectra_labels.to_numpy()
spectra_y1 = spectra_y.to_numpy()
spectra_labels1.shape

(11865, 17)

In [63]:
spectra_y3 = np.stack(spectra_y1)
spectra_y3.shape

(11865, 1800)

In [64]:
#Разбиение на трейн,тест и вал
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(spectra_y3, spectra_labels1, test_size=0.33, random_state=42, shuffle = True)

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_jobs=-1, random_state=0, verbose=1, warm_start=True)
clf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished


RandomForestClassifier(n_jobs=-1, random_state=0, verbose=1, warm_start=True)

In [66]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.7s finished


In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names = classes))

                  precision    recall  f1-score   support

          alkane       0.85      0.91      0.88      2670
          methyl       0.83      0.91      0.87      2431
          alkene       0.94      0.22      0.35       539
          alkyne       1.00      0.25      0.40        67
        alcohols       0.93      0.61      0.74       864
          amines       0.89      0.30      0.45       526
        nitriles       0.00      0.00      0.00       160
       aromatics       0.89      0.96      0.92      2270
   alkyl halides       0.81      0.39      0.53      1017
          esters       0.90      0.66      0.76       456
         ketones       0.99      0.18      0.30       379
       aldehydes       1.00      0.10      0.18        83
carboxylic acids       0.86      0.45      0.59       220
           ether       0.94      0.62      0.75       985
    acyl halides       1.00      0.03      0.05        38
          amides       0.87      0.12      0.21       269
           ni

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.


# Конец